# Worksheet 04

Name: Yiran Huang
UID: U71885452

### Topics

- Distance & Similarity
- Cost Functions
- K means

### Distance & Similarity

#### Part 1

a) In the minkowski distance, describe what the parameters p and d are.

d is demension, which represents the space that is d-dimensional
p is a parameter that makes the formula more flexible, which is greater than 1, as value of p changing, the formula can calculate different types of distance.

b) In your own words describe the difference between the Euclidean distance and the Manhattan distance.

According to the formula of minkowski distance,
p = 2 Euclidean distance
p = 1 Manhattan distance

Consider A = (0, 0) and B = (1, 1). When:

- p = 1, d(A, B) = 2
- p = 2, d(A, B) = $\sqrt{2} = 1.41$
- p = 3, d(A, B) = $2^{1/3} = 1.26$
- p = 4, d(A, B) = $2^{1/4} = 1.19$

c) Describe what you think distance would look like when p is very large.

distance will be much smaller as p is increasing

d) Is the minkowski distance still a distance function when p < 1? Expain why / why not.

no, in the triangle example with three points, the minkowski distance with p < 1 will violate the triangle inequality

e) when would you use cosine similarity over the euclidan distance?

when direction matters more than magnitude
use cosine similarity when direction is important and magnitude is not, and use Euclidean distance when both direction and magnitude are important.

f) what does the jaccard distance account for that the manhattan distance doesn't?

the size of the intersection

#### Part 2

Consider the following two sentences:

In [2]:
s1 = "hello my name is Alice"  
s2 = "hello my name is Bob"

using the union of words from both sentences, we can represent each sentence as a vector. Each element of the vector represents the presence or absence of the word at that index.

In this example, the union of words is ("hello", "my", "name", "is", "Alice", "Bob") so we can represent the above sentences as such:

In [3]:
v1 = [1,    1, 1,   1, 1,    0]
#     hello my name is Alice
v2 = [1,    1, 1,   1, 0, 1]
#     hello my name is    Bob

Programmatically, we can do the following:

In [11]:
corpus = [s1, s2]
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)
v1 = [1 if x in s1 else 0 for x in all_words]
print(v1)

['Bob', 'name', 'my', 'is', 'Alice', 'hello']
[0, 1, 1, 1, 1, 1]


Let's add a new sentence to our corpus:

In [12]:
s3 = "hi my name is Claude"
corpus.append(s3)

a) What is the new union of words used to represent s1, s2, and s3?

In [13]:
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)

['Bob', 'name', 'hi', 'Claude', 'my', 'is', 'Alice', 'hello']


b) Represent s1, s2, and s3 as vectors as above, using this new set of words.

In [17]:
v1 = [1 if x in s1 else 0 for x in all_words]
v2 = [1 if x in s2 else 0 for x in all_words]
v3 = [1 if x in s3 else 0 for x in all_words]
print(v1)
print(v2)
print(v3)

[0, 1, 0, 0, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 1, 0, 0]


c) Write a function that computes the manhattan distance between two vectors. Which pair of vectors are the most similar under that distance function?

In [38]:
def manhattan_dist(x, y):
    res = 0
    for i in range(len(x)):
        res += abs(x[i] - y[i])
    return res

print(manhattan_dist(v1, v2))
print(manhattan_dist(v1, v3))
print(manhattan_dist(v3, v2))

2
4
4


v1 and v2 are most similar since the distance is the shortest.

d) Create a matrix of all these vectors (row major) and add the following sentences in vector form:

- "hi Alice"
- "hello Claude"
- "Bob my name is Claude"
- "hi Claude my name is Alice"
- "hello Bob"

In [39]:
sentences = ["hi Alice", "hello Claude", "Bob my name is Claude", "hi Claude my name is Alice", "hello Bob"]
all_words = list(set([item for x in sentences for item in x.split()]))
print(all_words)
def create_vector(l):
    return [1 if x in l else 0 for x in all_words]
v = []
for s in sentences:
    v.append(create_vector(s))
print(v)

['Bob', 'name', 'hi', 'Claude', 'my', 'is', 'Alice', 'hello']
[[0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1], [1, 1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 1, 0], [1, 0, 0, 0, 0, 0, 0, 1]]


e) How many rows and columns does this matrix have?

In [40]:
print(len(v), ",", len(v[0]))

5 , 8


f) When using the Manhattan distance, which two sentences are the most similar?

In [41]:
for i in range(len(v)):
    for j in range(i + 1, len(v)):
        print(manhattan_dist(v[i], v[j]), " ", i + 1, "and", j + 1)

4   1 and 2
7   1 and 3
4   1 and 4
4   1 and 5
5   2 and 3
6   2 and 4
2   2 and 5
3   3 and 4
5   3 and 5
8   4 and 5


this two:
- "hello Claude"
- "hello Bob"

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. Initialize the centroids: The initial centroids are given as [0, 2].

2. Assign data points to the nearest centroid: We calculate the Euclidean distance between each data point and the two centroids. Points closest to centroid [0] will be assigned to cluster 1, and points closest to centroid [2] will be assigned to cluster 2.

3. Recalculate the centroids: We calculate the mean of all the points assigned to each cluster, and use this mean as the new centroid. So, if the points assigned to cluster 1 are [0, .5, 1.5], the new centroid for cluster 1 will be (0 + .5 + 1.5) / 3 = 1.0. Similarly, if the points assigned to cluster 2 are [2, 6, 6.5, 7], the new centroid for cluster 2 will be (2 + 6 + 6.5 + 7) / 4 = 5.375.

4. Repeat steps 2 and 3 until the centroids do not change anymore: If the new centroids are different from the previous iteration, we repeat steps 2 and 3. If the centroids stop changing, the algorithm terminates.

5. Return the final centroids: The final centroids are the optimal centroids that minimize the sum of squared distances between data points and their nearest centroid. In this example, the final centroids would be [1.0, 5.375].

b) Describe in plain english what the cost function for k means is.

Way to evaluate and compare solutions

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

1. Initial centroids: The initial centroids that are randomly selected can play a major role in determining the final cluster assignments. Different initial centroids can result in different final cluster solutions.

2. Data distribution: The distribution of data points within the dataset can result in different final solutions. For example, if the data points are densely packed in one area and sparse in another, different initial centroids may lead to different final solutions.

3. Number of iterations: The algorithm stops when the centroids are no longer changing, but the number of iterations performed can affect the final solution.

4. Distance metric: The distance metric used to determine the closest centroid to a data point can also affect the final solution. For example, Euclidean distance may produce different results compared to Manhattan distance.

d) Does Lloyd's Algorithm always converge? Why / why not?

The convergence of Lloyd's Algorithm is not guaranteed and depends on the choice of initial centroids and the shape and distribution of the data. In some cases, the algorithm may converge to a local minimum of the WCSS objective function instead of the global minimum, leading to suboptimal clustering results.

The algorithm may also get stuck in an infinite loop, oscillating between two points without converging to a final solution. This can happen if the centroids are updated such that the algorithm always alternates between two sets of centroids, and never reaches a steady state.